In [1]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# EOS토큰을 PAD토큰으로 지정하여 warning이 나오지 않게 함 warnings
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

2022-09-28 11:52:16.560552: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2e:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-28 11:52:16.595357: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2e:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-28 11:52:16.595797: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2e:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-09-28 11:52:16.597030: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

#### 1. GPT-2 Greedy Search로 디코딩하기

In [18]:
# 생성할 텍스트에 대한 시작 문구를 지정
input_ids = tokenizer.encode('I enjoy walking with my cute dog', return_tensors='tf')

# 디코딩 최대 길이(50글자)까지 model.generate를 통해 디코딩 진행
greedy_output = model.generate(input_ids, max_length=50)

print(greedy_output.shape)
for x in greedy_output:
  print("Output:\n" + 100 * '-')
  print(tokenizer.decode(x, skip_special_tokens=True))

(1, 50)
Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my dog.

I'm not sure if I'll


#### 2. Beam Search

In [19]:
# Beam-search를 적용하고 EOS가 나오면 생성을 멈춤
beam_output = model.generate(
    input_ids, 
    max_length=50, 
    num_beams=5, #1보다 큰 값을 지정
    early_stopping=True #EOS토큰이 나오면 생성을 중단
)

print(beam_output.shape)
print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

(1, 50)
Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I'm not sure if I'll ever be able to walk with him again. I'm not sure if I'll


#### 2.1 n-gram 패널티 주기

- 이 전략을 이용하면 훨씬 자연스러운 문장을 생성할 수 있지만, 텍스트 전체에서 n-gram으로 설정한 단어가 한 번만 등장할 수 있기 때문에 주의해서 사용해야 한다. 예를 들어 '서울 시청'에 대한 주제로 글을 쓰는데, no_repeat_ngram_size = 2로 설정하면 전체 글에서 '서울 시청'이라는 말은 한 번밖에 사용하지 못하기 때문이다!

In [20]:
beam_output = model.generate(
    input_ids, 
    max_length=50, 
    num_beams=5, 
    no_repeat_ngram_size=2, # 2-gram의 어구가 반복되지 않도록 설정함
    early_stopping=True
)

print(beam_output.shape)
print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

(1, 50)
Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to take a break


#### 2.2 beam search에서 k개의 beam을 모두 리턴하기

beam을 유지하며 디코딩한 시퀀스 중 가장 높은 확률을 가지는 k의 시퀀스를 모두 리턴해 마음에 드는 것을 사용하는 전략

- 코드에서는 num_return_sequences 옵션을 통해 구현 가능. 이 때 이 값은 num_beams보다 작거나 같아야 한다.

하지만, 모델이 자유롭게 글을 생성하는 <open-ended 생성>에서는 beam search가 최선의 전략은 아닐 수 있다:

beam search는 기계번역이나 요약정도에는 잘 작동하지만, 생성해야 하는 텍스트의 길이가 긴 대화 혹은 스토리를 생성해야 하는 open-ended 생성에서는 좋지 않다는 연구 결과가 있다.
beam search는 동어반복 문제가 심한 편인데, n-gram 페널티 전략으로는 '반복 없음'과 '적절한 시점에 동일한 단어를 재사용'하는 중간 지점을 찾기 어렵다. 
인간이 사용하는 언어를 놓고 보면, 모델이 생각하기에 가장 높은 확률을 가지는 단어가 늘 다음에 오는 것은 아니다. 마치 사람이 쓴 것과 같이 자연스럽기 위해서는 너무 예측 가능한 나머지 뻔하지만은 않은, '놀라운' 단어를 생성해낼 필요도 있다.

In [21]:
beam_outputs = model.generate(
    input_ids, 
    max_length=50, 
    num_beams=5, 
    no_repeat_ngram_size=2, 
    num_return_sequences=5, # 다섯 개의 문장을 리턴
    early_stopping=True
)

print(beam_outputs.shape)
for x in beam_outputs:
  print("Output:\n" + 100 * '-')
  print(tokenizer.decode(x, skip_special_tokens=True))

(5, 50)
Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to take a break
Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to get back to
Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with her again.

I've been thinking about this for a while now, and I think it's time for me to take a break
Output:
-------------------------------------------------------------------------------------------

#### 3. Sampling

: 모델이 생각하는 다음에 올 토큰에 대한 확률분포에 따라 단어를 샘플링하는 방식으로 디코딩하는 전략
- 이 전략을 사용하면 각 타임스텝에서 모델이 예측한 토큰의 확률분포를 이용해 토큰을 샘플링해 문장을 완성한다.

In [25]:
tf.random.set_seed(0) # reproductibility를 위해 랜덤 시드 고정

sample_output = model.generate(
    input_ids, 
    do_sample=True, # 샘플링 사용
    max_length=50, 
    top_k=0 #top_k=0으로 설정하면 타임스텝별로 하나의 토큰만 샘플링
)

print(sample_output.shape)
print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

(1, 50)
Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog because of myself.

Bryan: Oh, it's edgy.

Rachel: Does your dog love weddings?

Bryan: He won't, because he wants to

Rachel:


- 하지만 모델이 만들어낸 확률은 smooth한 나머지, 낮은 확률의 토큰이 "지나치게 잘" 샘플링될 수 있고, 이렇게 되면 어색한 문장이 만들어질 수 있다.
- 따라서 모델이 배출한 분포에서 높은 값을 가지는 확률을 더 뾰족하게 만드는 temperature 스케일링을 사용한다.
- 코드에서는 temperature 옵션을 조절하여 확률의 sharpness를 조절할 수 있고, temperature이 0에 가까워질수록 greedy decoding에 가까운 아웃풋이 나온다.

#### 4. Top-k Sampling

: 가장 확률이 높은 K개의 '다음 단어들'을 필터링하고, 확률 질량을 해당 K개의 '다음 단어들'에 대해 재분배하는 전략

- 이는 GPT-2에서 선택한 디코딩 전략으로, 스토리 생성에서 큰 효과를 보인 방법이다. 
- 코드상에서는 top_k 옵션을 0이 아닌 50과 같은 숫자로 주어 top-k 샘플링을 쉽게 구현할 수 있다.

- 단, 이 방법은 다음 토큰으로 필터링된 k의 단어를 아주 효과적으로 활용하지 못 할 수 있다는 우려가 있다. 그림에서 보여주는 예시만 보아도, 첫 번째 단어는 꽤나 평평한 분포에서 샘플링을 하지만 두 번째 토큰은 sharp한 분포에서 샘플링을 하게 된다. 이로 인해 첫 번째 타임스텝에서는 꽤나 괜찮아보이는 (people, big, house, cat) 등의 후보는 전혀 고려되지 못하고, 두 번째 타임스텝에서는 낮은 확률이라도 뽑게 되면 어색해지는 (down, a) 등의 토큰이 샘플링 풀에 포함되게 된다. 즉, 이 방법은 모델의 창의성을 지나치게 저하하면서도 모델이 이상한 단어를 샘플링할 위험이 있는 것이다.

#### 5. 그래서 끝판왕(?), Top-p (nucleus) sampling
: 가능도 있는 k개의 단어로부터 샘플링하는 대신, 누적 확률이 확률 p에 다다르는 최소한의 단어 집합으로부터 샘플링

- 가장 높은 확률을 가지는 토큰부터 시작해, 확률 값의 합이 top-p로 설정한 값을 넘을 때까지 샘플링 풀에 토큰을 추가한다.

In [27]:
tf.random.set_seed(0)

# deactivate top_k sampling and sample only from 92% most likely words
sample_output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=50, 
    top_p=0.92, #92%로 설정하고 샘플링하기
    top_k=0
)

print(sample_output.shape)
print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

(1, 50)
Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog because of the security and safety, and she wasn't scared by the obstacles. But so do her two grown-ups and now she's over scared by the amount of metal there is."

It might seem


- 이론상으로는 top-p 샘플링이 더 좋아보이지만, top-k와 top-p 샘플링 전략 모두 실제로 잘 작동한다.

- 또한, 두 전략을 섞어서 사용하면, 너무 낮게 랭킹된 토큰을 사용하는 것은 피하면서도 꽤나 다양한 시퀀스를 생성할 수 있다.

In [32]:
# 생성할 텍스트에 대한 시작 문구를 지정
#input_ids = tokenizer.encode('나는 지금 너무 배고픔을 느껴서 ', return_tensors='tf')
# 생성할 텍스트에 대한 시작 문구를 지정
input_ids = tokenizer.encode('I enjoy walking with my cute dog', return_tensors='tf')

sample_outputs = model.generate(
    input_ids,
    do_sample=True, #샘플링 전략 사용
    max_length=50, # 최대 디코딩 길이는 50
    top_k=50, # 확률 순위가 50위 밖인 토큰은 샘플링에서 제외
    top_p=0.95, # 누적 확률이 95%인 후보집합에서만 생성
    num_return_sequences=3 #3개의 결과를 디코딩해낸다
)

print(sample_outputs.shape)
for x in sample_outputs:
  print("Output:\n" + 100 * '-')
  print(tokenizer.decode(x, skip_special_tokens=True))

(3, 50)
Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog and even my mom is pretty into this type of thing but I guess I just have to give her the benefit of the doubt as I don't have any dog friends or social network. So there's a chance I
Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog. When I walk, I can tell that you are looking very thoughtful and compassionate," he said, adding that even though he is now a full-time school principal, she is still a dedicated caretaker for the
Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but after years of living in a house with a dog, the stress is getting too much to manage. I have been told we should adopt it before it hurts us.

We think it makes sense,


In [1]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

article_text = """Videos that say approved vaccines are dangerous and cause autism, cancer or infertility are among those that will be taken down, the company said.  The policy includes the termination of accounts of anti-vaccine influencers.  Tech giants have been criticised for not doing more to counter false health information on their sites.  In July, US President Joe Biden said social media platforms were largely responsible for people's scepticism in getting vaccinated by spreading misinformation, and appealed for them to address the issue.  YouTube, which is owned by Google, said 130,000 videos were removed from its platform since last year, when it implemented a ban on content spreading misinformation about Covid vaccines.  In a blog post, the company said it had seen false claims about Covid jabs "spill over into misinformation about vaccines in general". The new policy covers long-approved vaccines, such as those against measles or hepatitis B.  "We're expanding our medical misinformation policies on YouTube with new guidelines on currently administered vaccines that are approved and confirmed to be safe and effective by local health authorities and the WHO," the post said, referring to the World Health Organization."""

article_text = """

 우선 이미지에서 큰 특징을 추출한 다음 점차 작고 복잡한 특징을 발견해 나가는 방식이다. 예컨대 사진 속에 자동차가 있다고 해 보자. 알고리즘은 우선 사물의 전체적인 윤곽을 먼저 확인한 뒤 기존에 입력된 사진 데이터와 비교해 ‘탈 것’으로 범위를 좁힌다. 이후 타이어나 제조사 엠블럼처럼 세부적인 특징을 파악하고 ‘사진 속에 있는 물체는 자동차’라는 결론을 내리게 된다. 

"""
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

input_ids = tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    max_length=84,
    no_repeat_ngram_size=2,
    num_beams=4
)[0]

summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)

/home/tsyi/miniconda3/envs/cpu/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
2022-10-05 11:30:01.825950: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-05 11:30:02.041594: E tensorflow/stream_exec

사진 속에 있는 모든 물체의 특징을 파악하는 알고리즘을 소개한다.


In [10]:
tokenizer.cache_dir()

AttributeError: 'T5TokenizerFast' object has no attribute 'cache_dir'